In [1]:
import numpy as np
from torch import nn
import torch
from torchvision import models, transforms, datasets
import torch.nn.functional as F
import pretrainedmodels
import os

In [2]:
import pandas as pd

In [3]:
pretrained_model = {'resnet34' : '/home/artem-nb/Projects/DCL/models/pretrained/resnet34-333f7ec4.pth',}

import pdb

In [60]:
class LoadConfig(object):
    def __init__(self, args, version):
        if version == 'train':
            get_list = ['train', 'val']
        elif version == 'val':
            get_list = ['val']
        elif version == 'test':
            get_list = ['test']
        else:
            raise Exception("train/val/test ???\n")

        ###############################
        #### add dataset info here ####
        ###############################

        # put image data in $PATH/data
        # put annotation txt file in $PATH/anno

        if args.dataset == 'product':
            self.dataset = args.dataset
            self.rawdata_root = './../FGVC_product/data'
            self.anno_root = './../FGVC_product/anno'
            self.numcls = 2019
        elif args.dataset == 'CUB':
            self.dataset = args.dataset
            self.rawdata_root = './dataset/CUB_200_2011/data'
            self.anno_root = './dataset/CUB_200_2011/anno'
            self.numcls = 200
        elif args.dataset == 'STCAR':
            self.dataset = args.dataset
            self.rawdata_root = './dataset/st_car/data'
            self.anno_root = './dataset/st_car/anno'
            self.numcls = 196
        elif args.dataset == 'AIR':
            self.dataset = args.dataset
            self.rawdata_root = './dataset/aircraft/data'
            self.anno_root = './dataset/aircraft/anno'
            self.numcls = 100
        elif args.dataset == 'custom':
            self.dataset = args.dataset
            self.rawdata_root = ''
            self.anno_root = '/home/artem-nb/Datasets/coco_alcohol/annotations'
            self.numcls = 1660
        else:
            raise Exception('dataset not defined ???')

        # annotation file organized as :
        # path/image_name cls_num\n

        if 'train' in get_list:
             self.train_anno = pd.read_csv(os.path.join(self.anno_root, 'ct_train.txt'),\
                                           sep=" ",\
                                           header=None,\
                                           names=['ImageName', 'label'])

        if 'val' in get_list:
            self.val_anno = pd.read_csv(os.path.join(self.anno_root, 'ct_val.txt'),\
                                           sep=" ",\
                                           header=None,\
                                           names=['ImageName', 'label'])

        if 'test' in get_list:
            self.test_anno = pd.read_csv(os.path.join(self.anno_root, 'ct_test.txt'),\
                                           sep=" ",\
                                           header=None,\
                                           names=['ImageName', 'label'])

        self.swap_num = args.swap_num

        self.save_dir = './net_model'
        if not os.path.exists(self.save_dir):
            os.mkdir(self.save_dir)
        self.backbone = args.backbone

        self.use_dcl = True
        self.use_backbone = False if self.use_dcl else True
        self.use_Asoftmax = False
        self.use_focal_loss = False
        self.use_fpn = False
        self.use_hier = False

        self.weighted_sample = False
        self.cls_2 = True
        self.cls_2xmul = False

        self.log_folder = './logs'
        if not os.path.exists(self.log_folder):
            os.mkdir(self.log_folder)

In [61]:
m = pretrainedmodels.resnet34(num_classes=1000)

In [62]:
m.avgpool = nn.Sequential()
m.fc = nn.Sequential()
m.last_linear = nn.Sequential()

In [63]:
m

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [144]:
class MainModel(nn.Module):
    def __init__(self, config):
        super(MainModel, self).__init__()
        self.use_dcl = config.use_dcl
        self.num_classes = config.numcls
        self.backbone_arch = config.backbone
        self.use_Asoftmax = config.use_Asoftmax
        print(self.backbone_arch)

        if self.backbone_arch in dir(models):
            self.model = getattr(models, self.backbone_arch)()
            if self.backbone_arch in pretrained_model:
                self.model.load_state_dict(torch.load(pretrained_model[self.backbone_arch]))
        else:
            if self.backbone_arch in pretrained_model:
                self.model = pretrainedmodels.__dict__[self.backbone_arch](num_classes=1000, pretrained=None)
            else:
                self.model = pretrainedmodels.__dict__[self.backbone_arch](num_classes=1000)

        if self.backbone_arch in ['resnet34', 'resnet50', 'se_resnet50']:
            self.model = nn.Sequential(*list(self.model.children())[:-2])
            #self.model.avgpool = nn.Sequential()
            #self.model.fc = nn.Sequential()
            #self.model.last_linear = nn.Sequential()
        if self.backbone_arch == 'senet154':
            self.model = nn.Sequential(*list(self.model.children())[:-3])
        if self.backbone_arch == 'se_resnext101_32x4d':
            self.model = nn.Sequential(*list(self.model.children())[:-2])
        if self.backbone_arch == 'se_resnet101':
            self.model = nn.Sequential(*list(self.model.children())[:-2])
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=1)
        self.classifier = nn.Linear(2048, self.num_classes, bias=False)

        if self.use_dcl:
            if config.cls_2:
                self.classifier_swap = nn.Linear(2048, 2, bias=False)
            if config.cls_2xmul:
                self.classifier_swap = nn.Linear(2048, 2*self.num_classes, bias=False)
            self.Convmask = nn.Conv2d(2048, 1, 1, stride=1, padding=0, bias=True)
            self.avgpool2 = nn.AvgPool2d(1, stride=1)

        if self.use_Asoftmax:
            self.Aclassifier = AngleLinear(2048, self.num_classes, bias=False)

    def forward(self, x, last_cont=None):
        x = self.model(x)
        print(x.shape)
        if self.use_dcl:
            mask = self.Convmask(x)
            mask = self.avgpool2(mask)
            mask = torch.tanh(mask)
            mask = mask.view(mask.size(0), -1)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        out = []
        out.append(self.classifier(x))

        if self.use_dcl:
            out.append(self.classifier_swap(x))
            out.append(mask)

        if self.use_Asoftmax:
            if last_cont is None:
                x_size = x.size(0)
                out.append(self.Aclassifier(x[0:x_size:2]))
            else:
                last_x = self.model(last_cont)
                last_x = self.avgpool(last_x)
                last_x = last_x.view(last_x.size(0), -1)
                out.append(self.Aclassifier(last_x))

        return out

In [145]:
from easydict import EasyDict as edict

In [146]:
args = dict(auto_resume=False, backbone='resnet34', use_dcl=True, base_lr=0.0008, check_point=5000, cls_2=False, cls_lr_ratio=10.0, cls_mul=True, crop_resolution=[448, 448], dataset='custom', decay_step=60, discribe='training_descibe', epoch=360, resize_resolution=[512, 512], resume=None, save_point=5000, start_epoch=0, swap_num=[3, 3], train_batch=8, train_num_workers=8, val_batch=512, val_num_workers=16)

In [147]:
args = edict(args)

In [149]:
args

{'auto_resume': False,
 'backbone': 'resnet34',
 'use_dcl': True,
 'base_lr': 0.0008,
 'check_point': 5000,
 'cls_2': False,
 'cls_lr_ratio': 10.0,
 'cls_mul': True,
 'crop_resolution': [448, 448],
 'dataset': 'custom',
 'decay_step': 60,
 'discribe': 'training_descibe',
 'epoch': 360,
 'resize_resolution': [512, 512],
 'resume': None,
 'save_point': 5000,
 'start_epoch': 0,
 'swap_num': [3, 3],
 'train_batch': 8,
 'train_num_workers': 8,
 'val_batch': 512,
 'val_num_workers': 16}

In [150]:
config = LoadConfig(args, 'train')

In [151]:
model =  MainModel(config)

resnet34


In [161]:
model.model[7]

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BasicBlock(
    (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1

In [125]:
from torchsummary import summary

In [126]:
summary(model.model, (3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [162]:
model = nn.DataParallel(model)

In [164]:
model.module.model[7]

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BasicBlock(
    (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1

In [128]:
# model

In [129]:
layer0_param = list(map(id, model.module.model.conv1.parameters()))
layer0_param += list(map(id, model.module.model.bn1.parameters()))
layer1_param = list(map(id, model.module.model.layer1.parameters()))
layer2_param = list(map(id, model.module.model.layer2.parameters()))
layer3_param = list(map(id, model.module.model.layer3.parameters()))
layer4_param = list(map(id, model.module.model.layer4.parameters()))

In [130]:
# custom_lr_params = layer0_param + layer1_param + layer2_param + layer3_param + layer4_param
# base_lr_params = filter(lambda p: id(p) not in custom_lr_params, self.model.parameters())

In [131]:
# self.optimizer = torch.optim.SGD([
#     {'params': self.model.module.conv1.parameters(), 'lr': self.learning_rate * self.layer0_coef},
#     {'params': self.model.module.bn1.parameters(), 'lr': self.learning_rate * self.layer0_coef},
#     {'params': self.model.module.layer1.parameters(), 'lr': self.learning_rate * self.layer1_coef},
#     {'params': self.model.module.layer2.parameters(), 'lr': self.learning_rate * self.layer2_coef},
#     {'params': self.model.module.layer3.parameters(), 'lr': self.learning_rate * self.layer3_coef},
#     {'params': self.model.module.layer4.parameters(), 'lr': self.learning_rate * self.layer4_coef},
#     {'params': base_lr_params, 'lr': self.learning_rate},
# ], self.learning_rate,
#     momentum=self.momentum,
#     weight_decay=self.weight_decay)

In [132]:
ignored_params1 = list(map(id, model.module.classifier.parameters()))
ignored_params2 = list(map(id, model.module.classifier_swap.parameters()))
ignored_params3 = list(map(id, model.module.Convmask.parameters()))

In [133]:
ignored_params1

[140047723832232]

In [134]:
ignored_params2

[140047723834032]

In [135]:
ignored_params3

[140047723835040, 140047723832736]

In [136]:
ignored_params = ignored_params1 + ignored_params2 + ignored_params3 + layer0_param + layer1_param + layer2_param + layer3_param + layer4_param
print('the num of new layers:', len(ignored_params), flush=True)

the num of new layers: 112


In [137]:
base_params = filter(lambda p: id(p) not in ignored_params, model.module.parameters())

In [138]:
layer0_coef = 1e-3
layer1_coef = 1e-2
layer2_coef = 1e-2
layer3_coef = 1e-1
layer4_coef = 1e-1

In [139]:
from torch import optim

In [140]:
lr_ratio = args.cls_lr_ratio
base_lr = args.base_lr
# if args.use_backbone:
#     optimizer = optim.SGD([{'params': base_params},
#                            {'params': model.module.classifier.parameters(), 'lr': base_lr}], lr = base_lr, momentum=0.9)
# else:
optimizer = optim.SGD([{'params': base_params},
                       {'params': model.module.model.conv1.parameters(), 'lr': base_lr * layer0_coef},
                       {'params': model.module.model.bn1.parameters(), 'lr': base_lr * layer0_coef},
                       {'params': model.module.model.layer1.parameters(), 'lr': base_lr * layer1_coef},
                       {'params': model.module.model.layer2.parameters(), 'lr': base_lr * layer2_coef},
                       {'params': model.module.model.layer3.parameters(), 'lr': base_lr * layer3_coef},
                       {'params': model.module.model.layer4.parameters(), 'lr': base_lr * layer4_coef},
                       {'params': model.module.classifier.parameters(), 'lr': lr_ratio*base_lr},
                       {'params': model.module.classifier_swap.parameters(), 'lr': lr_ratio*base_lr},
                       {'params': model.module.Convmask.parameters(), 'lr': lr_ratio*base_lr},
                      ], lr = base_lr, momentum=0.9)

In [141]:
model

DataParallel(
  (module): MainModel(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): Batch

In [142]:
summary(model, (3, 448, 448), device='cpu')

torch.Size([2, 100352])


RuntimeError: Expected 4-dimensional input for 4-dimensional weight 1 2048 1 1, but got 2-dimensional input of size [2, 100352] instead

In [143]:
res224 = model(torch.rand(1, 3, 224, 224))

torch.Size([1, 25088])


RuntimeError: Expected 4-dimensional input for 4-dimensional weight 1 2048 1 1, but got 2-dimensional input of size [1, 25088] instead

In [15]:
res224[0].shape, res224[1].shape, res224[2].shape

(torch.Size([1, 1660]), torch.Size([1, 2]), torch.Size([1, 49]))

In [16]:
res448 = model(torch.rand(1, 3, 448, 448))

In [17]:
res448[0].shape, res448[1].shape, res448[2].shape

(torch.Size([1, 1660]), torch.Size([1, 2]), torch.Size([1, 196]))